In [17]:
import pandas as pd

# Загрузка исходных данных из файла parquet
data_cleaned = pd.read_parquet('data/data_cleaned.parquet')
data_cleaned['Год'] = pd.to_numeric(data_cleaned['Год'], errors='coerce')

columns_to_drop = [       
  ' Выручка, RUB',
' Краткосрочные обязательства, RUB', ' Кредиторская задолженность, RUB',
' Нераспределенная прибыль (непокрытый убыток), RUB',
' Себестоимость продаж, RUB', ' Собственный оборотный капитал, RUB',
]
data_cleaned = data_cleaned.drop(columns=columns_to_drop)


# Список столбцов, для которых нужно рассчитать изменения в процентах за последний год
columns_to_calculate = [' Cooтношение дебиторской задолженности к активам компании, %',
       ' Доля рабочего капитала в активах компании, %',
       ' Коэффициент абсолютной ликвидности, %',
       ' Коэффициент быстрой ликвидности, %',
       ' Коэффициент обеспеченности собственными оборотными средствами, %',
       ' Коэффициент оборачиваемости совокупных активов, %',
       ' Коэффициент соотношения заемных и собственных средств, %',
       ' Коэффициент текущей ликвидности, %',
       ' Период оборота активов, дни', ' Период оборота запасов, дни',
       ' Период оборота основных средств, дни',
       ' Период погашения дебиторской задолженности, дни',
       ' Период погашения кредиторской задолженности, дни',
       ' Рентабельность активов (ROA), %', ' Рентабельность затрат, %',
       ' Рентабельность капитала (ROE), %', ' Рентабельность продаж, %',
       ' Соотношение чистого долга к капиталу, %']

# Выбор последних двух годов для каждой компании
last_years_data = data_cleaned.groupby('Наименование').apply(lambda x: x.nlargest(2, 'Год')).reset_index(drop=True)

def calculate_percentage_change(df, columns, years):
    if len(df) < years:
        return pd.Series([float('NaN')] * len(columns), index=columns)
    changes = (df.iloc[years-1][columns] - df.iloc[0][columns]) / df.iloc[0][columns] * 100
    return changes

# Вычисление изменений в процентах за последние 2 и 3 года для указанных столбцов
last_2_years_changes = data_cleaned.groupby('Наименование').apply(calculate_percentage_change, columns=columns_to_calculate, years=2)
last_3_years_changes = data_cleaned.groupby('Наименование').apply(calculate_percentage_change, columns=columns_to_calculate, years=3)
last_4_years_changes = data_cleaned.groupby('Наименование').apply(calculate_percentage_change, columns=columns_to_calculate, years=4)

# Замена пропусков на нули
last_2_years_changes = last_2_years_changes.fillna(0)
last_3_years_changes = last_3_years_changes.fillna(0)
last_4_years_changes = last_4_years_changes.fillna(0)
# Переименование столбцов
last_2_years_changes.columns = [col + '_change_last_2_years' for col in last_2_years_changes.columns]
last_3_years_changes.columns = [col + '_change_last_3_years' for col in last_3_years_changes.columns]
last_4_years_changes.columns = [col + '_change_last_4_years' for col in last_4_years_changes.columns]


# Объединение информации об изменениях с исходным датафреймом
data_cleaned = pd.merge(data_cleaned, last_2_years_changes, on='Наименование', how='left')
data_cleaned = pd.merge(data_cleaned, last_3_years_changes, on='Наименование', how='left')
data_cleaned = pd.merge(data_cleaned, last_4_years_changes, on='Наименование', how='left')
# Вывод результата
data_cleaned


,index,Наименование,Регистрационный номер,Адрес (место нахождения),Код налогоплательщика,Регион регистрации,Вид деятельности/отрасль,Важная информация,Мои списки,Дата иска,...,"Период оборота активов, дни_change_last_4_years","Период оборота запасов, дни_change_last_4_years","Период оборота основных средств, дни_change_last_4_years","Период погашения дебиторской задолженности, дни_change_last_4_years","Период погашения кредиторской задолженности, дни_change_last_4_years","Рентабельность активов (ROA), %_change_last_4_years","Рентабельность затрат, %_change_last_4_years","Рентабельность капитала (ROE), %_change_last_4_years","Рентабельность продаж, %_change_last_4_years","Соотношение чистого долга к капиталу, %_change_last_4_years"
0,0,"100РА ЛТД, ООО",1.135038e+12,"119121, г. Москва, вн.тер.г. муниципальный окр...",5.038096e+09,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,"100РА ЛТД, ООО",1.135038e+12,"119121, г. Москва, вн.тер.г. муниципальный окр...",5.038096e+09,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,"100РА ЛТД, ООО",1.135038e+12,"119121, г. Москва, вн.тер.г. муниципальный окр...",5.038096e+09,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,12,"1015 УСМР, АО",1.027739e+12,"143003, Московская обл, Одинцово г, г.о. Одинц...",7.734009e+09,Московская область,Строительство жилых и нежилых зданий,\nЕсть заявление в суд о банкротстве (13.07.20...,"""STR""",2017-01-24,...,7.526882,300.0,500.000000,129.090909,-1.338238,108.333333,-1.234568,129.556650,-1.234568,-85.682551
4,13,"1015 УСМР, АО",1.027739e+12,"143003, Московская обл, Одинцово г, г.о. Одинц...",7.734009e+09,Московская область,Строительство жилых и нежилых зданий,\nЕсть заявление в суд о банкротстве (13.07.20...,"""STR""",2017-01-24,...,7.526882,300.0,500.000000,129.090909,-1.338238,108.333333,-1.234568,129.556650,-1.234568,-85.682551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39936,77274,"ЯСУ, ООО",1.137604e+12,"150040, Ярославская обл, г.о. город Ярославль,...",7.604248e+09,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,169.000000,12300.0,-87.899543,450.000000,169.021793,-77.857143,-481.142857,-80.977667,-515.697674,92.627509
39937,77276,"ЯСУ, ООО",1.137604e+12,"150040, Ярославская обл, г.о. город Ярославль,...",7.604248e+09,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,169.000000,12300.0,-87.899543,450.000000,169.021793,-77.857143,-481.142857,-80.977667,-515.697674,92.627509
39938,77277,"ЯСУ, ООО",1.137604e+12,"150040, Ярославская обл, г.о. город Ярославль,...",7.604248e+09,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,169.000000,12300.0,-87.899543,450.000000,169.021793,-77.857143,-481.142857,-80.977667,-515.697674,92.627509
39939,77278,"ЯСУ, ООО",1.137604e+12,"150040, Ярославская обл, г.о. город Ярославль,...",7.604248e+09,Ярославская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,169.000000,12300.0,-87.899543,450.000000,169.021793,-77.857143,-481.142857,-80.977667,-515.697674,92.627509


In [18]:
# Фильтруем данные по категориальной переменной
bankrupt_data = data_cleaned[data_cleaned['Категориальная переменная'] == True]
no_bankrupt_data = data_cleaned[data_cleaned['Категориальная переменная'] == False]

# Приводим столбец 'Год' в тип int для компаний
no_bankrupt_data['Год'] = no_bankrupt_data['Год'].astype(int)
bankrupt_data['Год'] = bankrupt_data['Год'].astype(int)

/tmp/ipykernel_2255/3680836658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_bankrupt_data['Год'] = no_bankrupt_data['Год'].astype(int)
/tmp/ipykernel_2255/3680836658.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bankrupt_data['Год'] = bankrupt_data['Год'].astype(int)


In [19]:
# Вычисляем год иска для компаний, которые банкроты
bankrupt_data['Год иска'] = bankrupt_data['Дата иска'].apply(lambda x: x.year - 2 if x.month > 3 else x.year - 1)

# Фильтруем банкроты по году
filtered_data = bankrupt_data[bankrupt_data['Год'] == bankrupt_data['Год иска']]
filtered_data

/tmp/ipykernel_2255/467618231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bankrupt_data['Год иска'] = bankrupt_data['Дата иска'].apply(lambda x: x.year - 2 if x.month > 3 else x.year - 1)


,index,Наименование,Регистрационный номер,Адрес (место нахождения),Код налогоплательщика,Регион регистрации,Вид деятельности/отрасль,Важная информация,Мои списки,Дата иска,...,"Период оборота запасов, дни_change_last_4_years","Период оборота основных средств, дни_change_last_4_years","Период погашения дебиторской задолженности, дни_change_last_4_years","Период погашения кредиторской задолженности, дни_change_last_4_years","Рентабельность активов (ROA), %_change_last_4_years","Рентабельность затрат, %_change_last_4_years","Рентабельность капитала (ROE), %_change_last_4_years","Рентабельность продаж, %_change_last_4_years","Соотношение чистого долга к капиталу, %_change_last_4_years",Год иска
7,16,"1015 УСМР, АО",1.027739e+12,"143003, Московская обл, Одинцово г, г.о. Одинц...",7.734009e+09,Московская область,Строительство жилых и нежилых зданий,\nЕсть заявление в суд о банкротстве (13.07.20...,"""STR""",2017-01-24,...,3.000000e+02,500.000000,129.090909,-1.338238,108.333333,-1.234568,129.556650,-1.234568,-85.682551,2016
43,80,"2МЕН ГРУПП, АО",1.067746e+12,"625002, Тюменская обл., г. Тюмень, ул. Комсомо...",7.701651e+09,Тюменская область,Строительство жилых и нежилых зданий,\nЕсть заявление в суд о банкротстве (03.08.20...,"""STR""",2019-10-21,...,-3.381087e+01,5063.636364,38.550390,-52.154827,-86.046512,-26.565875,-94.316731,-20.333220,47.508942,2017
77,130,"54, ООО ПМК",1.020203e+12,"450032, Башкортостан Респ, г.о. город Уфа, г У...",2.770551e+08,Башкортостан (Республика),Строительство жилых и нежилых зданий,\n\n\n,"""STR""",2020-10-14,...,5.107527e+01,11.864407,148.076923,45.798529,-602.173913,1320.833333,-1173.043478,1275.000000,-50.692690,2018
101,172,"А КОНСТРАКШН, ООО",1.147847e+12,"197183, г. Санкт-Петербург, вн.тер.г. муниципа...",7.814608e+09,Санкт-Петербург,Строительство жилых и нежилых зданий,\nЕсть заявление в суд о банкротстве (04.02.20...,"""STR""",2020-10-26,...,inf,inf,-57.446809,-14.717310,-33.642578,-63.567251,-2.357574,-59.890486,-96.478914,2018
130,237,"А.С.СТРОЙ, ООО",1.074706e+12,"187330, Ленинградская обл, м.р-н Кировский, г....",4.706027e+09,Ленинградская область,Строительство жилых и нежилых зданий,\n\nНа 09.10.2023 17:02 есть решения ФНС о при...,"""STR""",2017-08-07,...,1.024286e+04,133.333333,771.428571,3784.653465,-95.033712,-41.813602,-83.772455,-39.945652,307.802341,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39839,77085,"ЮТА, ООО СК",1.138617e+12,"628452, Ханты-Мансийский Автономный Округ - Юг...",8.617031e+09,Тюменская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",2021-05-17,...,2.341667e+03,4277.777778,-10.937500,32.658859,-85.321101,-59.345794,-148.324022,-58.851675,-361.826885,2019
39876,77150,"ЯИК, ООО",1.125027e+12,"140011, Московская обл., г. Люберцы, ул. Авиат...",5.027184e+09,Московская область,Строительство жилых и нежилых зданий,Юридическое лицо признано несостоятельным (бан...,"""STR""",2021-06-01,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2019
39908,77225,"ЯНТАРЬСЕРВИСБАЛТИК, ООО",1.103925e+12,", Калининградская обл, г.о. город Калининград,...",3.906221e+09,Калининградская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",2017-02-07,...,inf,-100.000000,-3.349282,42.812284,8.108108,1000.000000,-17.906951,911.111111,2550.000000,2016
39917,77236,"ЯРАВТОДОР, ООО",1.147602e+12,"150506, Ярославская обл., Ярославский район, п...",7.602111e+09,Ярославская область,Строительство жилых и нежилых зданий,\nСообщение о признаках банкротства (17.10.202...,"""STR""",2020-11-02,...,1.035000e+04,inf,325.000000,279.450906,-81.386861,182.432432,-91.422647,171.889401,-92.838115,2018


In [20]:
# # Удаляем строки с отсутствующими значениями в данных
# filtered_data = filtered_data.dropna()
# no_bankrupt_data = no_bankrupt_data.dropna()

# Создаем dataframe с парами компаний
pairs = []
used_companies = set()

for index, row in filtered_data.iterrows():
    year = row['Год']
    assets = row[' Активы  всего, RUB']  # Предположим, что у вас есть столбец 'Активы всего, RUB'

    # Фильтруем данные не банкротов для текущего года
    candidates = no_bankrupt_data[(no_bankrupt_data['Год'] == year)]
    if not candidates.empty:
        # Находим ближайшую компанию по размеру активов
        nearest_index = (candidates[' Активы  всего, RUB'] - assets).abs().idxmin()
        nearest_company_name = no_bankrupt_data.loc[nearest_index, 'Наименование']
        # Проверяем, использовали ли уже компанию в предыдущих парах
        if nearest_company_name not in used_companies:
            # Расчитываем разницу в процентах между банкротной и ближайшей компаниями
            bankrupt_assets = filtered_data.at[index, ' Активы  всего, RUB']
            nearest_assets = no_bankrupt_data.at[nearest_index, ' Активы  всего, RUB']
            percent_difference = ((nearest_assets - bankrupt_assets) / bankrupt_assets) * 100

            # Создаем пару и добавляем в список
            pair = {
                'Банкротная компания ID': index,
                'Ближайшая компания ID': nearest_index,
                'Год': year,
                'Разница в %': percent_difference,
            }
            pairs.append(pair)

            # Добавляем компанию в использованные
            used_companies.add(nearest_company_name)

# Создаем dataframe из списка пар
pairs_df = pd.DataFrame(pairs)
pairs_df

,Банкротная компания ID,Ближайшая компания ID,Год,Разница в %
0,7,19809,2016,0.092112
1,43,21173,2017,-1.028048
2,77,22991,2018,-0.065555
3,101,20356,2018,-0.025952
4,130,26194,2015,-0.007693
...,...,...,...,...
643,39763,11998,2018,-0.289716
644,39839,3101,2019,0.043269
645,39876,11123,2019,0.002211
646,39917,17823,2018,-0.009404


In [21]:

# Извлекаем строки из оригинальных датафреймов по индексам
bankrupt_companies = filtered_data.loc[pairs_df['Банкротная компания ID']]
nearest_companies = no_bankrupt_data.loc[pairs_df['Ближайшая компания ID']]

# Объединяем информацию из оригинальных датафреймов
matched_data = pd.concat([nearest_companies, bankrupt_companies])

# Удаляем все дубликаты по наименованию компании
matched_data = matched_data.drop_duplicates(subset='Наименование', keep=False)
matched_data

,index,Наименование,Регистрационный номер,Адрес (место нахождения),Код налогоплательщика,Регион регистрации,Вид деятельности/отрасль,Важная информация,Мои списки,Дата иска,...,"Период оборота запасов, дни_change_last_4_years","Период оборота основных средств, дни_change_last_4_years","Период погашения дебиторской задолженности, дни_change_last_4_years","Период погашения кредиторской задолженности, дни_change_last_4_years","Рентабельность активов (ROA), %_change_last_4_years","Рентабельность затрат, %_change_last_4_years","Рентабельность капитала (ROE), %_change_last_4_years","Рентабельность продаж, %_change_last_4_years","Соотношение чистого долга к капиталу, %_change_last_4_years",Год иска
19809,39123,"ПЭТРУСКО, ООО",1.137746e+12,"119285, г. Москва, вн.тер.г. муниципальный окр...",7.730683e+09,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,-6.360424,50.000000,-36.842105,-1.420407,48.920863,92.327150,-60.030738,75.188679,-94.956860,NaN
21173,41643,"РОССТРОЙИНВЕСТ, ЗАО",1.037828e+12,"197198, г. Санкт-Петербург, проспект Добролюбо...",7.813161e+09,Санкт-Петербург,Строительство жилых и нежилых зданий,Прекратило деятельность при слиянии (07.12.2022).,"""STR""",1970-01-01,...,-23.103058,71.698113,2583.673469,211.195944,-85.714286,103.061224,-99.175824,95.490716,-101.842543,NaN
22991,44832,"СЕРВИССТРОЙ, ООО",1.178617e+12,"628680, Ханты-Мансийский Автономный Округ - Юг...",8.605030e+09,Тюменская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,-57.983193,-84.109589,-78.443114,-64.032633,279.940120,-253.803749,281.889764,-222.768305,-71.460177,NaN
20356,40177,"РЕЛЬЕФ, ООО",1.127847e+12,"195299, г. Санкт-Петербург, проспект Гражданск...",7.804493e+09,Санкт-Петербург,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,3.174603,inf,2220.000000,663.830629,-89.541473,68.020305,-74.774716,65.803109,-32.841632,NaN
26194,50733,"СК-СТРОЙ, ООО",1.085405e+12,"630132, Новосибирская обл, г.о. город Новосиби...",5.405375e+09,Новосибирская область,Строительство жилых и нежилых зданий,\n\n\n,"""А1"", ""STR""",1970-01-01,...,161.463415,-32.653061,37.500000,2.544224,-91.562500,-70.737913,-90.078983,-69.841270,-86.407153,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39763,76909,"ЮНИТЕЛ ГРУПП, ООО",5.067746e+12,"108811, г. Москва, км. Киевское Шоссе 22-Й (П ...",7.727585e+09,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",2020-05-06,...,219.807692,-57.894737,88.557214,68.456450,-41.250000,576.000000,-37.799043,551.351351,-97.811218,2018.0
39839,77085,"ЮТА, ООО СК",1.138617e+12,"628452, Ханты-Мансийский Автономный Округ - Юг...",8.617031e+09,Тюменская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",2021-05-17,...,2341.666667,4277.777778,-10.937500,32.658859,-85.321101,-59.345794,-148.324022,-58.851675,-361.826885,2019.0
39876,77150,"ЯИК, ООО",1.125027e+12,"140011, Московская обл., г. Люберцы, ул. Авиат...",5.027184e+09,Московская область,Строительство жилых и нежилых зданий,Юридическое лицо признано несостоятельным (бан...,"""STR""",2021-06-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2019.0
39917,77236,"ЯРАВТОДОР, ООО",1.147602e+12,"150506, Ярославская обл., Ярославский район, п...",7.602111e+09,Ярославская область,Строительство жилых и нежилых зданий,\nСообщение о признаках банкротства (17.10.202...,"""STR""",2020-11-02,...,10350.000000,inf,325.000000,279.450906,-81.386861,182.432432,-91.422647,171.889401,-92.838115,2018.0


In [22]:
cols_to_drop = matched_data.filter(like='Активы').columns
matched_data = matched_data.drop(columns=cols_to_drop)
# Save the matching_data DataFrame as a Parquet file
matched_data.to_parquet('data/matched_data.parquet')

# Save the matching_data DataFrame as an Excel file
matched_data.to_excel('data/matched_data.xlsx', index=False)
matched_data

,index,Наименование,Регистрационный номер,Адрес (место нахождения),Код налогоплательщика,Регион регистрации,Вид деятельности/отрасль,Важная информация,Мои списки,Дата иска,...,"Период оборота запасов, дни_change_last_4_years","Период оборота основных средств, дни_change_last_4_years","Период погашения дебиторской задолженности, дни_change_last_4_years","Период погашения кредиторской задолженности, дни_change_last_4_years","Рентабельность активов (ROA), %_change_last_4_years","Рентабельность затрат, %_change_last_4_years","Рентабельность капитала (ROE), %_change_last_4_years","Рентабельность продаж, %_change_last_4_years","Соотношение чистого долга к капиталу, %_change_last_4_years",Год иска
19809,39123,"ПЭТРУСКО, ООО",1.137746e+12,"119285, г. Москва, вн.тер.г. муниципальный окр...",7.730683e+09,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,-6.360424,50.000000,-36.842105,-1.420407,48.920863,92.327150,-60.030738,75.188679,-94.956860,NaN
21173,41643,"РОССТРОЙИНВЕСТ, ЗАО",1.037828e+12,"197198, г. Санкт-Петербург, проспект Добролюбо...",7.813161e+09,Санкт-Петербург,Строительство жилых и нежилых зданий,Прекратило деятельность при слиянии (07.12.2022).,"""STR""",1970-01-01,...,-23.103058,71.698113,2583.673469,211.195944,-85.714286,103.061224,-99.175824,95.490716,-101.842543,NaN
22991,44832,"СЕРВИССТРОЙ, ООО",1.178617e+12,"628680, Ханты-Мансийский Автономный Округ - Юг...",8.605030e+09,Тюменская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,-57.983193,-84.109589,-78.443114,-64.032633,279.940120,-253.803749,281.889764,-222.768305,-71.460177,NaN
20356,40177,"РЕЛЬЕФ, ООО",1.127847e+12,"195299, г. Санкт-Петербург, проспект Гражданск...",7.804493e+09,Санкт-Петербург,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",1970-01-01,...,3.174603,inf,2220.000000,663.830629,-89.541473,68.020305,-74.774716,65.803109,-32.841632,NaN
26194,50733,"СК-СТРОЙ, ООО",1.085405e+12,"630132, Новосибирская обл, г.о. город Новосиби...",5.405375e+09,Новосибирская область,Строительство жилых и нежилых зданий,\n\n\n,"""А1"", ""STR""",1970-01-01,...,161.463415,-32.653061,37.500000,2.544224,-91.562500,-70.737913,-90.078983,-69.841270,-86.407153,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39763,76909,"ЮНИТЕЛ ГРУПП, ООО",5.067746e+12,"108811, г. Москва, км. Киевское Шоссе 22-Й (П ...",7.727585e+09,Москва,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",2020-05-06,...,219.807692,-57.894737,88.557214,68.456450,-41.250000,576.000000,-37.799043,551.351351,-97.811218,2018.0
39839,77085,"ЮТА, ООО СК",1.138617e+12,"628452, Ханты-Мансийский Автономный Округ - Юг...",8.617031e+09,Тюменская область,Строительство жилых и нежилых зданий,\n\n\n,"""STR""",2021-05-17,...,2341.666667,4277.777778,-10.937500,32.658859,-85.321101,-59.345794,-148.324022,-58.851675,-361.826885,2019.0
39876,77150,"ЯИК, ООО",1.125027e+12,"140011, Московская обл., г. Люберцы, ул. Авиат...",5.027184e+09,Московская область,Строительство жилых и нежилых зданий,Юридическое лицо признано несостоятельным (бан...,"""STR""",2021-06-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2019.0
39917,77236,"ЯРАВТОДОР, ООО",1.147602e+12,"150506, Ярославская обл., Ярославский район, п...",7.602111e+09,Ярославская область,Строительство жилых и нежилых зданий,\nСообщение о признаках банкротства (17.10.202...,"""STR""",2020-11-02,...,10350.000000,inf,325.000000,279.450906,-81.386861,182.432432,-91.422647,171.889401,-92.838115,2018.0
